<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/%EC%B5%9C%EC%A2%85code/MLM_%EC%A6%9D%EB%B6%84_%EA%B3%A0%EB%8F%84%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [22]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 25.2 MB/s eta 0:00:00


In [23]:
from transformers import pipeline, AutoTokenizer
from konlpy.tag import Okt
import random

def mask_and_predict(sentence, model_name="beomi/KcBERT-base", num_masks=2, top_k=5):

    # 형태소 분석기(Okt) 사용하여 단어 분리
    okt = Okt()
    words = okt.morphs(sentence)  # 형태소 단위로 분리

    # BERT 모델 및 토크나이저 로드
    mlm_pipeline = pipeline("fill-mask", model=model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 마스킹할 후보 단어 선택 (명사, 동사 우선)
    maskable_indices = [i for i, word in enumerate(words) if word not in ['을', '를', '이', '가', '은', '는', '에', '도', '의', '한', '가장']]

    # 마스킹할 단어 개수 설정
    num_masks = min(num_masks, len(maskable_indices))
    mask_indices = random.sample(maskable_indices, num_masks)

    # 마스킹 적용
    for idx in mask_indices:
        words[idx] = "[MASK]"

    # 마스킹된 문장 생성
    masked_sentence = " ".join(words)

    # MLM 모델로 예측 수행
    predictions = mlm_pipeline(masked_sentence, top_k=top_k)

    # 여러 개의 [MASK]를 예측하여 문장 복원
    new_sentence = masked_sentence
    for i, mask_preds in enumerate(predictions):
        best_pred = random.choice([pred["token_str"] for pred in mask_preds[:3]])  # Top 3 중 랜덤 선택
        new_sentence = new_sentence.replace("[MASK]", best_pred, 1)

    return new_sentence


In [25]:
# 테스트 실행
test_sentence = "지루한 진보와 헛된 결말 ... 단 5 분만에 평범한 가족 역사 이야기를 펼친 것 같습니다. 미래에 감독의 영화를 보지 않기로 결정한 사람."
print(mask_and_predict(test_sentence))

Some weights of the model checkpoint at beomi/KcBERT-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


지루한 진보 의 헛된 결말 ... 단 5 분만 에 평범한 가족 역사 이야기 를 펼친 것 같습니다 . 미래 에 감독 의 영화 를 보지 않기로 결정 한 ##겁니다 .


In [20]:
sentence="지루한 진보와 헛된 결말 ... 단 5 분만에 평범한 가족 역사 이야기를 펼친 것 같습니다. 미래에 감독의 영화를 보지 않기로 결정한 사람."
mask_and_predict(sentence, model_name="beomi/KcBERT-base", num_masks=5)

Some weights of the model checkpoint at beomi/KcBERT-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


'[CLS] 지루한 평화와 헛된 결말.. 이분은 단 5 분만에 평범한 가족 역사 이야기를 펼친 사람 같습니다. 나는 이상의 영화를 보지 않기로 결정한 사람. [SEP]'

# 데이터 불러오기

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'  'My Drive'   NSMC_0.02.pkl   NSMC_0.5.pkl   이전   졸업논문


In [6]:
import pandas as pd
df=pd.read_csv('/content/NSMC_0.8_증분대상_1000개.csv', encoding='utf-8-sig')

In [7]:
df['document'].isnull().sum()

0

In [8]:
df['MLM_aug'] = df['document'].apply(mask_and_predict)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at beomi/KcBERT-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

Device set to use cpu
Some weights of the model checkpoint at beomi/KcBERT-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Some weights of the model checkpoint at beomi/KcBERT-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing Bert

In [9]:
df.head(5)

,Unnamed: 0,id,document,label,MLM_aug
0,0,7652286,솔직히 이영화때문에 슈퍼맨시리즈가 7년이란 세월동안 묻혀있엇다 맨오브스틸은 미국에서...,0,[CLS] 솔직히 이영화때문에 영화맨시리즈가 7년이란 세월동안 묻혀있엇다 맨오브스틸...
1,1,8895340,아무리 짱짱맨 옵대장의 다때려부수는 CG가 좋다그래도 이딴 디워랑 견줄만한 스토리로...,0,[CLS] 아무리 짱짱맨 옵대장의 다때려부수는 CG가 많아도그래도 저 디워랑 견줄만...
2,2,6549317,오프닝장면 죠스영화 고대루 베겼네?? 여자 비명 연기하는 것도 고대루네~! 그냥 갖...,0,[CLS] 오프닝장면 죠스영화 고대루 베겼네?? 여자 비명 연기하는 것도 완전루네 ...
3,3,8650550,대표적인 친일선전영화라서 최하점을 줄수밖에 없다. 영화수준 자체도 초딩 학예회수준의...,0,[CLS] 대표적인 친일선전영화라서 그런점을 줄수밖에 없다. 영화수준 자체도 초딩 ...
4,4,8658070,아~ 그렇구나~ 남편이 두 여자랑 바람펴도 조강지처 잘못인거구나~ㅎㅎ 부재는 '회개...,0,[CLS] 아 ~ 그렇구나 ~ 남편이 두 여자랑 바람펴도 조강지처 잘못인거구나 ~ ...


In [10]:
df.to_csv('NSMC_0.8_aug_mlm_kcbert.csv', encoding='utf-8-sig')